<a href="https://colab.research.google.com/github/constistentviv28/project/blob/main/dogs_breed_prediction_using_cnn_inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import PIL
import cv2
import requests
from io import BytesIO
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.backend import clear_session

from tensorflow.keras.applications import InceptionResNetV2,InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.xception import Xception

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/Dog_Images_train_test_val/archive (2).zip','r')
zip_ref.extractall('/content')
zip_ref.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/Dog_Images_train_test_val/archive (2).zip'

In [ ]:
# define paths
DIMS = (224,224)
IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 42

In [ ]:
# generate datasets
# ImageDatagenerator to load the images in batches and perform data augmentation

data_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_data = data_gen.flow_from_directory(batch_size=BATCH_SIZE, directory='/content/train', shuffle=True,
                                            target_size=DIMS, class_mode='categorical')

val_data = data_gen.flow_from_directory(batch_size=BATCH_SIZE, directory='/content/valid', shuffle=True,
                                            target_size=DIMS, class_mode='categorical')

test_data = data_gen.flow_from_directory(batch_size=BATCH_SIZE, directory='/content/test', shuffle=True,
                                            target_size=DIMS, class_mode='categorical')

In [ ]:
label_mapper = np.asarray(list(train_data.class_indices.keys()))
label_mapper

In [ ]:
IMG_SHAPE = (224,224,3)
base_model = InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet', classes=70)

In [ ]:
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())
model.add(Dense(70,activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=2, factor=0.001)
es = EarlyStopping(monitor='val_loss', verbose=2, patience=5, min_delta=0.001)
# checkpoint = ModelCheckpoint(filepath='/kaggle/working/model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
logs = model.fit(train_data, validation_data=val_data,
                steps_per_epoch = train_data.samples//BATCH_SIZE,
                validation_steps = val_data.samples//BATCH_SIZE,
                epochs=10, verbose=1, callbacks=[reduce_lr,es])

In [ ]:
clear_session()

In [ ]:
plt.plot(logs.history['loss'], label='loss')
plt.plot(logs.history['val_loss'], label='val_loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.title('Loss Logs')
plt.show()

In [ ]:

plt.plot(logs.history['accuracy'], label='accuracy', linewidth=2)
plt.plot(logs.history['val_accuracy'], label='val_accuracy', linewidth=2)
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.title('Accuracy Logs')
plt.show()

In [ ]:
model.evaluate(val_data)

In [ ]:
model.evaluate(test_data)

In [ ]:
def predictor(img):
    display(img)
    arr = img_to_array(img)
    arr = arr/255.0
    arr = np.expand_dims(arr,0)
    res = model.predict(arr)
    print(res.shape)
    idx = res.argmax()
    return label_mapper[idx], res[0][idx]

In [ ]:
# requests.get(image_url, stream=True).raw
url = 'https://cdn.britannica.com/16/234216-050-C66F8665/beagle-hound-dog.jpg'
response = requests.get(url)
img = load_img(BytesIO(response.content), target_size=DIMS)

val,prob = predictor(img)
val,prob

In [ ]:
# save the model
os.chdir('/content/gdrive/MyDrive/Dog_Images_train_test_val/Dog_Model_save')
model.save('model.keras')